Question 5a

In [1]:
using DataFrames, CSV, LinearAlgebra, StatsModels, Plots, GLM


oil = CSV.read("oil-well-drilling-costs.csv", DataFrame)
fo = @formula(Cost ~ 1 + Depth)
A = modelmatrix(fo.rhs, oil)
b = vec(modelmatrix(fo.lhs, oil))



16-element Vector{Float64}:
  2596.8
  3328.0
  3181.1
  3198.4
  4779.9
  5905.6
  5769.2
  8089.5
  4813.1
  5618.7
  7736.0
  6788.3
  7840.8
  8882.5
 10489.5
 12506.6

Reading in the oil data, and creating the model matrix and response vector

Part B


In [2]:
f(x) = 0.5 * norm(A*x - b)^2 #the objective function

∇f(x) = A'*A*x - A'*b # gradient
 # initial guess

∇f (generic function with 1 method)

This is the function to minimize the sum of squared errors, as well as the inital gradient my model matrix and response vector above.

Part C

In [3]:

H = A'*A #hessian
invH = inv(H)


2×2 Matrix{Float64}:
  0.756884    -8.08022e-5
 -8.08022e-5   9.40258e-9

Calculates the Hessian and its inverse

Part D

In [4]:

function steepest_descent(f,∇f, x; α = 0.000000001, ϵ = 1e-1, k = 100000)
    i = 1
    while norm(∇f(x)) > ϵ
        p = -∇f(x)

        x = x + α * p
        i % k == 0 && println("iteration ", ". x = ", x)
        i += 1

    end
    return x
end



steepest_descent (generic function with 1 method)

Calculates the steepest descent with a fixed step size of 0.01. It intakes the gradient calculates before, as well as our initial x that we create below. e is the tolerance for convergance. 

Part E

In [5]:

function newton_method(f,∇f, x, H; αfixed = 1, ϵ = 1e-1, k = 10)
    i = 1
    while norm(∇f(x)) > ϵ
        p = -inv(H) * ∇f(x)
        
        x = x + αfixed * p
        i % k == 0 && println("iteration ", ". x = ", x)
        i += 1

    end
    return x
end



newton_method (generic function with 1 method)

This is the newton method of approximation, it doesn't involve as many outputs as the steepest descent does. Its another important search direction, and you have to use the inverse Hessian for this one. Otherwise its very similar to steepest descent, with the fixed step size and tolerance for convergence. 

Part F

In [6]:
x0 = [-2200,0.5]
newton_method(f, ∇f, x0, H) 


2-element Vector{Float64}:
 -2277.069654638195
     1.0033390629260879

As you can see, Newton method immediately got the answer in no iterations (as opposed to steepest descent).

In [7]:
x0 = [-2200,0.5]
steepest_descent(f, ∇f, x0)

iteration . x = [-2200.010128971447, 0.9951124643022881]
iteration . x = [-2200.0203094543417, 0.9951135511341347]
iteration . x = [-2200.0304885920577, 0.9951146378223746]
iteration . x = [-2200.040666385107, 0.995115724367063]
iteration . x = [-2200.050842833554, 0.9951168107682064]
iteration . x = [-2200.0610179374776, 0.995117897025813]
iteration . x = [-2200.071191697347, 0.9951189831399335]
iteration . x = [-2200.081364112861, 0.9951200691105355]
iteration . x = [-2200.091535184785, 0.9951211549377004]
iteration . x = [-2200.101704912673, 0.9951222406213811]
iteration . x = [-2200.1118732972714, 0.9951233261616567]
iteration . x = [-2200.122040338325, 0.9951244115585005]
iteration . x = [-2200.1322060363173, 0.9951254968119639]
iteration . x = [-2200.1423703912315, 0.995126581922045]
iteration . x = [-2200.1525334033367, 0.9951276668887727]
iteration . x = [-2200.1626950728078, 0.9951287517121654]
iteration . x = [-2200.172855399747, 0.9951298363922345]
iteration . x = [-2200.183

LoadError: InterruptException:

These are the results for running steepest descent and newton method. As you can see, steepest descent needed an absurd amount of iterations before it converged - so many that I was cut off by "Excessive output truncated after 524304 bytes.". Otherwise, it just returned INF and NaN. 

Part G

In [8]:
using GLM
df = oil
linearmodel = lm(fo, df)

lm(fo, df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Cost ~ 1 + Depth

Coefficients:
────────────────────────────────────────────────────────────────────────────────
                   Coef.   Std. Error      t  Pr(>|t|)     Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────────
(Intercept)  -2277.07     765.499      -2.97    0.0100  -3918.9       -635.238
Depth            1.00334    0.0853205  11.76    <1e-07      0.820345     1.18633
────────────────────────────────────────────────────────────────────────────────

The steepest descent gets the right answer, but it takes so long to travel there that its completely inefficient and slow. Newton's method excel's on this problem since its able to find the curve of the function, so its skips the steepest descents method of slowly skipping down. Newton's is a much better method of finding convergence in this case.